## Text Analysis - Companies description 

In this notebook, we will do some text analysis for the companies existing in the WRDS and Harvard Business School, Impact-Weighted Accounts Project report.

We will apply some Nature Language Processing (NLP) using the pre-trained DistilBERT. First, we need to merge the datasets and obtain the companies description from Yahoo Finance. 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import warnings
from sklearn import linear_model
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import requests
from bs4 import BeautifulSoup
warnings.filterwarnings('ignore')

In [ ]:
df_ishares = pd.read_csv('/Users/maralinetorres/Documents/GitHub/Predicting-Environmental-and-Social-Actions/Datasets/iShares-merged.csv')
df_ei = pd.read_csv('/Users/maralinetorres/Documents/GitHub/Predicting-Environmental-and-Social-Actions/Datasets/Environmental_impact_cleaned.csv')
stocks = pd.read_csv("/Users/maralinetorres/Documents/GitHub/Predicting-Environmental-and-Social-Actions/Datasets/pilot_stocks.csv")

For the ishares, we don't need the tickers in the sector 'Cash and devirates' and we are also going to filter by the necessary columns

In [ ]:
df_ishares = df_ishares.loc[df_ishares.Sector != 'Cash and/or Derivatives',['Ticker','Name','Sector','CUSIP','ISIN','Location']]
df_ishares.drop_duplicates(inplace=True)
df_ishares.head()

,Ticker,Name,Sector,CUSIP,ISIN,Location
0,AAPL,APPLE INC,Information Technology,37833100,US0378331005,United States
1,MSFT,MICROSOFT CORP,Information Technology,594918104,US5949181045,United States
2,AMZN,AMAZON COM INC,Consumer Discretionary,23135106,US0231351067,United States
3,TSLA,TESLA INC,Consumer Discretionary,88160R101,US88160R1014,United States
4,FB,FACEBOOK CLASS A INC,Communication,30303M102,US30303M1027,United States


In [ ]:
len(df_ishares.Ticker.unique())

855

In [ ]:
df_ei2 = df_ei.copy()
df_ei2 = df_ei2.loc[:,['ISIN','CompanyName','Country']]
df_ei2.drop_duplicates(inplace=True)
df_ei2.head()

,ISIN,CompanyName,Country
0,GB00BMX64W89,Saga plc,United Kingdom
1,MYL1818OO003,BURSA MALAYSIA BHD,Malaysia
2,GB0031638363,INTERTEK GROUP PLC,United Kingdom
3,ZAE000079711,JSE LIMITED,South Africa
4,FR0006174348,BUREAU VERITAS SA,France


In [ ]:
len(df_ei2.ISIN.unique())

2628

In [ ]:
stocks = stocks.iloc[:,0:2]
stocks.head()

,Year,Ticker
0,2005,AEE
1,2006,AEE
2,2007,AEE
3,2008,AEE
4,2009,AEE


Let's start merging the datasets. First, we are going to merge iShares and Environmental impact by ISIN. 

In [ ]:
df = pd.merge(df_ishares, df_ei2, on='ISIN', how='outer')
df.head()

,Ticker,Name,Sector,CUSIP,ISIN,Location,CompanyName,Country
0,AAPL,APPLE INC,Information Technology,37833100,US0378331005,United States,NaN,NaN
1,MSFT,MICROSOFT CORP,Information Technology,594918104,US5949181045,United States,MICROSOFT CORPORATION,United States
2,AMZN,AMAZON COM INC,Consumer Discretionary,23135106,US0231351067,United States,"AMAZON.COM, INC.",United States
3,TSLA,TESLA INC,Consumer Discretionary,88160R101,US88160R1014,United States,NaN,NaN
4,FB,FACEBOOK CLASS A INC,Communication,30303M102,US30303M1027,United States,FACEBOOK INCORPORATION,United States


In [ ]:
print(df.shape)
print(len(df.Ticker.unique()))

(3005, 8)
856


In [ ]:
mising_Ticker = df.loc[df.Ticker.isna(),'ISIN']
df_missing = df_ei.loc[df_ei.ISIN.isin(mising_Ticker), ['ISIN','CompanyName','Country']]
df_missing.drop_duplicates(inplace=True)
print(f'We are unable to match {df_missing.shape[0]} ISIN')
df_missing.head()

We are unable to match 2136 ISIN


,ISIN,CompanyName,Country
0,GB00BMX64W89,Saga plc,United Kingdom
1,MYL1818OO003,BURSA MALAYSIA BHD,Malaysia
3,ZAE000079711,JSE LIMITED,South Africa
5,GB0007370074,RICARDO PLC,United Kingdom
6,AU000000ASX7,ASX LIMITED,Australia


With this merge, we were able to find a 492 tickers for the 2,628 companies in the Environmental Impact dataset. We need to figure out how to map their ISIN to a Ticker.

In [ ]:
df_missing.to_csv('Companies_missing_Tickers.csv', index=False)

In the meantime, we are going to export the companies that we were unable to match and continue working with the ones that have the Ticker.

From the pilot stocks, we have 52 unique tickers. Let's see if all these tickers appear in the merged dataframe. 

In [ ]:
tickers = stocks.Ticker.unique()
x = df.loc[(df.Ticker.isin(tickers)), ['Ticker']]
print(x.shape)
print(f'{len(x.Ticker.unique())} Tickers')
x[x.duplicated()]

(55, 1)
52 Tickers


,Ticker
540,DTE
726,MRO
802,ATO


We noticed that these three tickers are duplicated. Let's see why?

In [ ]:
df_ishares[df_ishares.Ticker == 'ATO']

,Ticker,Name,Sector,CUSIP,ISIN,Location
417,ATO,ATMOS ENERGY CORP,Utilities,49560105,US0495601058,United States
809,ATO,ATOS,Information Technology,S56547813,FR0000051732,France


It seems the ticker is not globally unique. We are going to filter by United States because our pilot stocks belongs to the SP500 US Stocks. 

In [ ]:
df_us = df.loc[(df.Ticker.isin(tickers)) & (df.Location == 'United States'), ['Ticker']]
print(f'Now, we have {len(df_us.Ticker.unique())} tickers.')

Now, we have 52 tickers.


We know that we weren't able to match the ISIN for all the companies in the Environmental Intensity dataset. However, let's see the ones that we were able too and validate they matched correctly. 

In [ ]:
found = df.loc[(df.Ticker.notna()) & (df.CompanyName.notna()),]
print(len(found.Ticker.unique()))
print(found.shape)

487
(495, 8)


Could it be possible to have duplicate values? Let's subset for the records that have the same ticker.

In [ ]:
ticker_agg = found.groupby('Ticker')[['Name']].count().sort_values(by='Name', ascending=False).reset_index()
tickers2 = ticker_agg[ticker_agg.Name >= 2]['Ticker']
df3 = found[found.Ticker.isin(tickers2)].sort_values(by='Ticker')
df3.head(6)

,Ticker,Name,Sector,CUSIP,ISIN,Location,CompanyName,Country
428,AAL,AMERICAN AIRLINES GROUP INC,Industrials,02376R102,US02376R1023,United States,AMERICAN AIRLINES GROUP INC,United States
557,AAL,ANGLO AMERICAN PLC,Materials,-,GB00B1XZS820,United Kingdom,ANGLO AMERICAN PLC,United Kingdom
99,ADP,AUTOMATIC DATA PROCESSING INC,Information Technology,53015103,US0530151036,United States,"AUTOMATIC DATA PROCESSING, INC.",United States
866,ADP,AEROPORTS DE PARIS SA,Industrials,-,FR0010340141,France,AEROPORTS DE PARIS,France
271,DTE,DTE ENERGY,Utilities,233331107,US2333311072,United States,DTE ENERGY COMPANY,United States
540,DTE,DEUTSCHE TELEKOM N AG,Communication,S58423591,DE0005557508,Germany,DEUTSCHE TELEKOM AG,Germany


It happened the same thing as before, it seems the Ticker is not globally unique. However, it seems it matches correctly by ISIN. We did a little research and found that AAL in UK actually is AAL.L in Yahoo Finance. 

Another example,  DTE- DEUTSCHE TELEKOM ticker in Yahoo Finance is DTE.DE.  We decided to re-format the Ticker for the companies that are not from US. To some of them, we need to add '.' + 'the first two letters from the ISIN' to the current ticker. 

Let's see. 

In [ ]:
ticker_agg = df.groupby('Ticker')[['Name']].count().sort_values(by='Name', ascending=False).reset_index()
tickers2 = ticker_agg[ticker_agg.Name >= 2]['Ticker']
df3 = df[df.Ticker.isin(tickers2)].sort_values(by='Ticker')
df3.head(6)

,Ticker,Name,Sector,CUSIP,ISIN,Location,CompanyName,Country
428,AAL,AMERICAN AIRLINES GROUP INC,Industrials,02376R102,US02376R1023,United States,AMERICAN AIRLINES GROUP INC,United States
557,AAL,ANGLO AMERICAN PLC,Materials,-,GB00B1XZS820,United Kingdom,ANGLO AMERICAN PLC,United Kingdom
735,ADM,ADMIRAL GROUP PLC,Financials,-,GB00B02J6398,United Kingdom,ADMIRAL GROUP PLC,United Kingdom
234,ADM,ARCHER DANIELS MIDLAND,Consumer Staples,39483102,US0394831020,United States,NaN,NaN
866,ADP,AEROPORTS DE PARIS SA,Industrials,-,FR0010340141,France,AEROPORTS DE PARIS,France
99,ADP,AUTOMATIC DATA PROCESSING INC,Information Technology,53015103,US0530151036,United States,"AUTOMATIC DATA PROCESSING, INC.",United States


In [ ]:
df4 = df3[df3.Location != 'United States']
df4

,Ticker,Name,Sector,CUSIP,ISIN,Location,CompanyName,Country
557,AAL,ANGLO AMERICAN PLC,Materials,-,GB00B1XZS820,United Kingdom,ANGLO AMERICAN PLC,United Kingdom
735,ADM,ADMIRAL GROUP PLC,Financials,-,GB00B02J6398,United Kingdom,ADMIRAL GROUP PLC,United Kingdom
866,ADP,AEROPORTS DE PARIS SA,Industrials,-,FR0010340141,France,AEROPORTS DE PARIS,France
802,ATO,ATOS,Information Technology,S56547813,FR0000051732,France,ATOS SE,France
553,DG,VINCI SA,Industrials,-,FR0000125486,France,VINCI,France
540,DTE,DEUTSCHE TELEKOM N AG,Communication,S58423591,DE0005557508,Germany,DEUTSCHE TELEKOM AG,Germany
560,EL,ESSILORLUXOTTICA SA,Consumer Discretionary,S72124779,FR0000121667,France,ESSILORLUXOTTICA SA,France
731,LEG,LEG IMMOBILIEN N AG,Real Estate,-,DE000LEG1110,Germany,LEG IMMOBILIEN AG,Germany
631,MRK,MERCK,Health Care,S47418447,DE0006599905,Germany,MERCK KGAA,Germany
726,MRO,MELROSE INDUSTRIES PLC,Industrials,-,GB00BZ1G4322,United Kingdom,MELROSE INDUSTRIES PLC,United Kingdom


In [ ]:
ticker = df4.Ticker.tolist()
yahoo_ticker = ['AAL.L','ADM.L','ADP.PA','ATO.PA','DG.PA','DTE.DE','EL.PA','LEG.DE','MRK.DE','MRO.L','PRU.L','SAN.PA','SAN.MC','TEL.OL','TSCO.L']

In [ ]:
df = df.sort_values(by='Ticker')
df.loc[(df.Ticker.isin(ticker)) & (df.Location != 'United States'), 'Ticker'] = yahoo_ticker
df.head()

,Ticker,Name,Sector,CUSIP,ISIN,Location,CompanyName,Country
867,-,IBERDROLA SA,Utilities,-,ES0144583236,Spain,NaN,NaN
711,1COV,COVESTRO AG,Materials,-,DE0006062144,Germany,COVESTRO AG,Germany
170,A,AGILENT TECHNOLOGIES INC,Health Care,00846U101,US00846U1016,United States,"AGILENT TECHNOLOGIES, INC.",United States
428,AAL,AMERICAN AIRLINES GROUP INC,Industrials,02376R102,US02376R1023,United States,AMERICAN AIRLINES GROUP INC,United States
557,AAL.L,ANGLO AMERICAN PLC,Materials,-,GB00B1XZS820,United Kingdom,ANGLO AMERICAN PLC,United Kingdom


In [ ]:
print(len(df.Ticker.unique()))

870


Now, we have 870 unique tickers that we can send to yahoo finance and get the company description. Below, we will create a tickers list. Also, we want to export the merged dataset to do some analysis onwards. 

In [ ]:
tickers = df.Ticker.unique().tolist()

In [ ]:
df.to_csv('df_merged.csv', index=False)

### Collecting companies data in Reuters website

Reuters, the news and media division of Thomson Reuters, is the world's largest multimedia news provider, reaching billions of people worldwide every day.

First, we are going to add '.OQ' to each ticker. T

In [ ]:
tickers_US = df.loc[(df.Location == 'United States') & (df.Ticker.notna()),'Ticker'].unique().tolist()
tickers_nonUS = df.loc[(df.Location != 'United States') & (df.Ticker.notna()),'Ticker'].unique().tolist()

In [ ]:
# reuter_ticker_nonUS = []
# for ticker in tickers_nonUS:
#     data = df.loc[df.Ticker == ticker, 'Location'].tolist()
#     new_ticker = str(ticker) + '.' + str(data[0][0])
#     reuter_ticker_nonUS.append(new_ticker)
# reuter_ticker_nonUS[:5]

['-.S', '1COV.G', 'AAL.L.U', 'ABBN.S', 'ABF.U']

In [ ]:
reuter_ticker = []
for ticket in tickers_US:
    ticket = str(ticket) + '.OQ'
    reuter_ticker.append(ticket)

In [ ]:
# Create a loop to store URLs of all stocks' description page
URL = [] 
DES = [] 
comp_desc = {}
for i in reuter_ticker:
    url ='https://www.reuters.com/companies/'+i 
    URL.append(url)
    page = requests.get(url) # visits the URL
    htmldata = BeautifulSoup(page.content, 'html.parser')
    Business_Description = htmldata.find('p',{'class':'TextLabel__text-label___3oCVw TextLabel__black___2FN-Z TextLabel__serif___3lOpX Profile-body-2Aarn'}) # finds the business description part in the HTML code
    if Business_Description is not None:
        DES.append(Business_Description.text)
        comp_desc[i] = [Business_Description.text]
    else:
        comp_desc[i] = 'Not exists'  

In [ ]:
exists = 0 
do_not = 0
for val in comp_desc.values():
    if val == 'Not exists':
        do_not+=1
    else:
        exists+=1

print(do_not)
print(exists)

362
143


### Collect companies descriptions from Yahoo Finance

In this section, we will grab the company description directly from the Yahoo Finance website for the 870 companies

In [ ]:
# Create a loop to store URLs of all stocks' description page
URL = [] 
DES = [] 
comp_desc = {}
for i in tickers[:2]: 
    i = str(i)
    url ='https://finance.yahoo.com/quote/'+i+'/profile' 
    URL.append(url)
    page = requests.get(url) # visits the URL 
    htmldata = BeautifulSoup(page.content, 'html.parser')
    Business_Description = htmldata.find('p',{'class':'Mt(15px) Lh(1.6)'}) # finds the business description part in the HTML code
    DES.append(Business_Description.text)
    comp_desc[i] = [Business_Description.text]

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
df_comp_desc = pd.DataFrame.from_dict(comp_desc, orient='index', columns = ['Description'])
df_comp_desc.reset_index(inplace = True)
df_comp_desc.rename(columns = {'index':'Ticker'}, inplace=True)
df_comp_desc.head()

,Ticker,Description
0,-,None
1,1COV,None
2,A,None
3,AAL,None
4,AAL.L,None


In [ ]:
df_comp_desc.to_csv('Companydescription.csv',index=False)

### Start working with the DistilBERT

If you don't have the package install, you might need to run: !pip install transformers

If you do, only import the packcage and continue working. 

# DistilBERT

Load, merge and clean the data

In [ ]:
# load the csv files
stock_des=pd.read_csv('202 companies_des.csv')
df = pd.read_csv('IEV_holdings-1.csv')
df1=pd.read_csv('EE-ISIN_merged.csv') 

In [ ]:
df1=df1.drop('Unnamed: 0',1)

In [ ]:
df1=df1[df1['Year']==2019]
df1

,ISIN,Year,CompanyName,Country,Industry(Exiobase),EnvironmentalIntensity(Sales),EnvironmentalIntensity(OpInc),TotalEnvironmentalCost,WorkingCapacity,FishProductionCapacity,CropProductionCapacity,MeatProductionCapacity,Biodiversity,AbioticResources,Waterproductioncapacity(Drinkingwater&IrrigationWater),WoodProductionCapacity,SDG1.5,SDG2.1,SDG2.2,SDG2.3,SDG2.4,SDG3.3,SDG3.4,SDG3.9,SDG6,SDG12.2,SDG14.1,SDG14.2,SDG14.3,SDG14.c,SDG15.1,SDG15.2,SDG15.5,%Imputed,gvkey,fyear,datadate,at,isin,conm,fic,sic
0,MYL1818OO003,2019,BURSA MALAYSIA BHD,Malaysia,Activities auxiliary to financial intermediati...,-0.017,-3.47%,"-1,968,379","-1,924,910",-451,"-25,349","-5,938",-81,-168,"-11,502",20,"-852,646","-502,708","-502,460","-6,337","-6,337","-81,118","-4,791",-27,"-11,502",-168,-1,-1,-222,-2,10,10,-79,4%,272691,2019,20191231,2321.040,MYL1818OO003,BURSA MALAYSIA BHD,MYS,6200.0
1,GB0031638363,2019,INTERTEK GROUP PLC,United Kingdom,Activities auxiliary to financial intermediati...,-0.015,-9.49%,"-60,599,272","-59,281,663","-13,774","-788,289","-184,802","-2,487","-3,804","-324,960",508,"-26,533,166","-15,557,810","-15,550,827","-197,072","-197,072","-2,509,207","284,215",-703,"-324,960","-3,804",-17,-4,"-6,861",-20,254,254,"-2,470",1%,252384,2019,20191231,2818.400,GB0031638363,INTERTEK GROUP PLC,GBR,8700.0
2,ZAE000079711,2019,JSE LIMITED,South Africa,Activities auxiliary to financial intermediati...,-0.015,NaN,"-2,290,124","-2,239,814",-510,"-29,662","-6,938",-93,-901,"-12,200",-6,"-995,881","-576,811","-576,488","-7,415","-7,415","-92,910","-19,470",-277,"-12,200",-901,0,-1,-253,0,-3,-3,-93,2%,278391,2019,20191231,40227.215,ZAE000079711,JSE LIMITED,ZAF,6211.0
3,FR0006174348,2019,BUREAU VERITAS SA,France,Activities auxiliary to financial intermediati...,-0.007,-5.10%,"-39,978,650","-39,107,612","-9,330","-520,701","-121,953","-1,671","-4,116","-214,438","1,172","-17,514,837","-10,430,409","-10,425,281","-130,175","-130,175","-1,684,676","561,195",-577,"-214,438","-4,116",-38,-9,"-4,607",-45,586,586,"-1,633",3%,286961,2019,20191231,7049.100,FR0006174348,BUREAU VERITAS SA,FRA,8700.0
4,GB0007370074,2019,RICARDO PLC,United Kingdom,Activities auxiliary to financial intermediati...,-0.007,-7.27%,"-3,247,235","-3,176,408",-753,"-42,228","-9,899",-135,-468,"-17,406",63,"-1,421,576","-842,731","-842,343","-10,557","-10,557","-136,060","34,998",-87,"-17,406",-468,-2,0,-373,-2,31,31,-133,3%,221859,2019,20190630,371.900,GB0007370074,RICARDO PLC,GBR,8711.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,ZAE000216537,2019,BID CORPORATION LTD,South Africa,"Wholesale trade and commission trade, except o...",-0.014,-28.02%,"-124,047,261","-121,344,740","-28,962","-1,603,616","-374,519","-5,177","-35,468","-658,523","3,743","-53,787,360","-32,057,343","-32,036,952","-400,904","-400,904","-5,177,518","533,888","-10,313","-658,523","-35,468",-167,-103,"-14,129",-200,"1,871","1,871","-5,007",6%,321670,2019,20190630,64931.978,ZAE000216537,BID CORP(NEW),ZAF,6799.0
1058,ZAE000058517,2019,SPAR GROUP LIMITED,South Africa,"Wholesale trade and commission trade, except o...",-0.003,-49.56%,"-24,127,362","-23,594,642","-6,350","-305,744","-70,543","-1,082","-27,940","-124,010","2,948","-10,145,768","-6,518,174","-6,510,343","-76,436","-76,436","-1,059,431","420,567","-8,124","-124,010","-27,940",-132,-81,"-2,898",-157,"1,474","1,474",-948,20%,271087,2019,20190930,34052.900,ZAE000058517,SPAR GROUP LTD,ZAF,5140.0
1059,CNE100000FN7,2019,SINOPHARM GROUP CO LTD,China,"Wholesale trade and commission trade, except o...",-0.001,-1.85%,"-46,263,408","-42,158,085","-10,358","-562,862","-131,780","-1,839","-16,404","-3,384,365","2,285","-18,933,076","-11,481,810","-11,476,273","-140,715","-140,715","-1,857,658","1,173,028",-694,"-3,384,365","-16,404",-77,-13,"-5,068",-92,"1,142","1,142","-1,761",1%,292783,2019,20191231,269888.371,CNE100000FN7,SINOPHARM GROU

In [ ]:
df2=pd.merge(df,df1,on='ISIN')
df2

,Ticker,Name,Sector,Asset Class,Market Value,Weight (%),Notional Value,Shares,CUSIP,ISIN,SEDOL,Price,Location,Exchange,Currency,FX Rate,Market Currency,Accrual Date,Year,CompanyName,Country,Industry(Exiobase),EnvironmentalIntensity(Sales),EnvironmentalIntensity(OpInc),TotalEnvironmentalCost,WorkingCapacity,FishProductionCapacity,CropProductionCapacity,MeatProductionCapacity,Biodiversity,AbioticResources,Waterproductioncapacity(Drinkingwater&IrrigationWater),WoodProductionCapacity,SDG1.5,SDG2.1,SDG2.2,SDG2.3,SDG2.4,SDG3.3,SDG3.4,SDG3.9,SDG6,SDG12.2,SDG14.1,SDG14.2,SDG14.3,SDG14.c,SDG15.1,SDG15.2,SDG15.5,%Imputed,gvkey,fyear,datadate,at,isin,conm,fic,sic
0,NESN,NESTLE SA,Consumer Staples,Equity,"64,181,630.45",3.32,"64,181,630.45","505,570.00",S71238703,CH0038863350,7123870,126.95,Switzerland,SIX Swiss Exchange,USD,0.90,CHF,-,2019,NESTLE S.A.,Switzerland,Processing of Food products nec,-0.016,-9.75%,"-1,527,139,399","-1,405,885,897","-416,410","-18,578,077","-4,368,093","-65,196","-113,252","-97,865,932","153,457","-628,976,294","-420,810,968","-420,563,568","-4,644,519","-4,644,519","-68,675,754","119,281,774","-6,492","-97,865,932","-113,252","-5,213",-135,"-201,854","-6,213","76,729","76,729","-59,918",0%,16603,2019,20191231,127940.000,CH0038863350,NESTLE SA/AG,CHE,2000.0
1,ROG,ROCHE HOLDING PAR AG,Health Care,Equity,"46,859,542.94",2.42,"46,859,542.94","123,210.00",S71103881,CH0012032048,7110388,380.32,Switzerland,SIX Swiss Exchange,USD,0.90,CHF,-,2019,ROCHE HOLDING AKTIENGESELLSCHAFT,Switzerland,"Manufacture of medical, precision and optical ...",-0.003,-0.84%,"-160,446,737","-152,869,527","-35,079","-1,974,118","-461,147","-6,303","-1,149,997","-3,950,618",52,"-66,193,394","-38,441,169","-38,420,015","-493,529","-493,529","-6,193,341","-4,364,957","-722,554","-3,950,618","-1,149,997",-139,-317,"-16,901",-165,26,26,"-6,163",6%,25648,2019,20191231,83091.000,CH0012032048,ROCHE HOLDING AG,CHE,2834.0
2,NOVN,NOVARTIS AG,Health Care,Equity,"40,415,079.41",2.09,"40,415,079.41","433,340.00",S71030654,CH0012005267,7103065,93.26,Switzerland,SIX Swiss Exchange,USD,0.90,CHF,-,2019,NOVARTIS AG,Switzerland,"Manufacture of medical, precision and optical ...",-0.007,-3.18%,"-348,286,772","-314,529,861","-74,014","-4,143,676","-967,262","-13,198","-903,525","-27,659,340","4,103","-138,890,514","-81,935,724","-81,878,242","-1,035,919","-1,035,919","-13,219,836","-1,114,082","-567,694","-27,659,340","-903,525",-246,-249,"-36,343",-294,"2,051","2,051","-12,948",3%,101310,2019,20191231,118370.000,CH0012005267,NOVARTIS AG,CHE,2834.0
3,AZN,ASTRAZENECA PLC,Health Care,Equity,"26,955,662.18",1.39,"26,955,662.18","230,130.00",S09895293,GB0009895292,989529,117.13,United Kingdom,London Stock Exchange,USD,0.71,GBP,-,2019,ASTRAZENECA PLC,United Kingdom,"Manufacture of medical, precision and optical ...",-0.005,-3.98%,"-134,985,600","-129,784,288","-39,655","-1,688,455","-393,446","-6,368","-573,484","-2,526,476","26,573","-56,694,491","-39,535,401","-39,506,085","-422,114","-422,114","-6,472,569","11,948,529","-781,929","-2,526,476","-573,484","-1,050",-312,"-18,122","-1,251","13,286","13,286","-5,305",22%,28272,2019,20191231,61377.000,GB0009895292,ASTRAZENECA PLC,GBR,2834.0
4,SIE,SIEMENS N AG,Industrials,Equity,"23,306,443.99",1.21,"23,306,443.99","141,729.00",S57279739,DE0007236101,5727973,164.44,Germany,Xetra,USD,0.82,EUR,-,2019,SIEMENS AG,Germany,Activities of membership organisation n.e.c. (91),-0.005,-5.49%,"-431,933,738","-405,755,192","-97,827","-5,290,306","-1,228,970","-16,859","-600,004","-18,945,339",758,"-176,454,055","-102,930,381","-102,840,656","-1,322,577","-1,322,577","-16,586,392","-10,700,011","-167,303","-18,945,339","-600,004",-206,"-1,140","-46,960",-246,379,379,"-16,650",7%,19349,2019,20190930,150248.000,DE0007236101,SIEMENS AG,DEU,9997.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [ ]:
df2=pd.merge(df,df1,on='ISIN')
df2 = pd.merge(df2, stock_des, on='Ticker',how='inner')
df2

,Ticker,Name,Sector,Asset Class,Market Value,Weight (%),Notional Value,Shares,CUSIP,ISIN,SEDOL,Price,Location,Exchange,Currency,FX Rate,Market Currency,Accrual Date,Year,CompanyName,Country,Industry(Exiobase),EnvironmentalIntensity(Sales),EnvironmentalIntensity(OpInc),TotalEnvironmentalCost,WorkingCapacity,FishProductionCapacity,CropProductionCapacity,MeatProductionCapacity,Biodiversity,AbioticResources,Waterproductioncapacity(Drinkingwater&IrrigationWater),WoodProductionCapacity,SDG1.5,SDG2.1,SDG2.2,SDG2.3,SDG2.4,SDG3.3,SDG3.4,SDG3.9,SDG6,SDG12.2,SDG14.1,SDG14.2,SDG14.3,SDG14.c,SDG15.1,SDG15.2,SDG15.5,%Imputed,gvkey,fyear,datadate,at,isin,conm,fic,sic,description
0,ROG,ROCHE HOLDING PAR AG,Health Care,Equity,"46,859,542.94",2.42,"46,859,542.94","123,210.00",S71103881,CH0012032048,7110388,380.32,Switzerland,SIX Swiss Exchange,USD,0.90,CHF,-,2019,ROCHE HOLDING AKTIENGESELLSCHAFT,Switzerland,"Manufacture of medical, precision and optical ...",-0.003,-0.84%,"-160,446,737","-152,869,527","-35,079","-1,974,118","-461,147","-6,303","-1,149,997","-3,950,618",52,"-66,193,394","-38,441,169","-38,420,015","-493,529","-493,529","-6,193,341","-4,364,957","-722,554","-3,950,618","-1,149,997",-139,-317,"-16,901",-165,26,26,"-6,163",6%,25648,2019,20191231,83091.000,CH0012032048,ROCHE HOLDING AG,CHE,2834.0,"Rogers Corporation designs, develops, manufact..."
1,NOVN,NOVARTIS AG,Health Care,Equity,"40,415,079.41",2.09,"40,415,079.41","433,340.00",S71030654,CH0012005267,7103065,93.26,Switzerland,SIX Swiss Exchange,USD,0.90,CHF,-,2019,NOVARTIS AG,Switzerland,"Manufacture of medical, precision and optical ...",-0.007,-3.18%,"-348,286,772","-314,529,861","-74,014","-4,143,676","-967,262","-13,198","-903,525","-27,659,340","4,103","-138,890,514","-81,935,724","-81,878,242","-1,035,919","-1,035,919","-13,219,836","-1,114,082","-567,694","-27,659,340","-903,525",-246,-249,"-36,343",-294,"2,051","2,051","-12,948",3%,101310,2019,20191231,118370.000,CH0012005267,NOVARTIS AG,CHE,2834.0,"Novan, Inc., a clinical development-stage biot..."
2,AZN,ASTRAZENECA PLC,Health Care,Equity,"26,955,662.18",1.39,"26,955,662.18","230,130.00",S09895293,GB0009895292,989529,117.13,United Kingdom,London Stock Exchange,USD,0.71,GBP,-,2019,ASTRAZENECA PLC,United Kingdom,"Manufacture of medical, precision and optical ...",-0.005,-3.98%,"-134,985,600","-129,784,288","-39,655","-1,688,455","-393,446","-6,368","-573,484","-2,526,476","26,573","-56,694,491","-39,535,401","-39,506,085","-422,114","-422,114","-6,472,569","11,948,529","-781,929","-2,526,476","-573,484","-1,050",-312,"-18,122","-1,251","13,286","13,286","-5,305",22%,28272,2019,20191231,61377.000,GB0009895292,ASTRAZENECA PLC,GBR,2834.0,"AstraZeneca PLC discovers, develops, manufactu..."
3,SAN,SANOFI SA,Health Care,Equity,"21,623,837.19",1.12,"21,623,837.19","201,397.00",S56717358,FR0000120578,5671735,107.37,France,Nyse Euronext - Euronext Paris,USD,0.82,EUR,-,2019,SANOFI S.A.,France,"Manufacture of medical, precision and optical ...",-0.010,-6.09%,"-434,457,155","-393,211,644","-77,490","-4,399,917","-1,003,233","-13,907","-451,590","-35,245,079","-54,294","-143,654,775","-75,667,077","-75,484,826","-1,099,979","-1,099,979","-12,050,491","-89,489,470","-108,652","-35,245,079","-451,590",-461,"-3,419","-33,072",-549,"-27,147","-27,147","-13,441",25%,101204,2019,20191231,112736.000,FR0000120578,SANOFI,FRA,2834.0,"Banco Santander, S.A., together with its subsi..."
4,SAN,BANCO SANTANDER SA,Financials,Equity,"12,744,703.60",0.66,"12,744,703.60","3,038,453.00",S57059461,ES0113900J37,5705946,4.19,Spain,Bolsa De Madrid,USD,0.82,EUR,-,2019,BANCO SANTANDER SA,Spain,"Financial intermediation, except insurance and...",-0.001,-0.70%,"-99,333,328","-92,801,728","-25,942","-1,243,952","-290,468","-4,367","-52,104","-4,928,087","13,320","-41,800,811","-27,420,552","-27,402,198","-310,988","-310,988","-4,466,925","7,367,610","-4,289","-4,928,087","-52,104",-461,-47,"-12,357",-549,"6,660","6,660","-3,901",13%,14140,2019,20191231,1522695.0

Create a binary variable that is 1 if the assets to revenue ratio is above its median and 0 otherwise.

This is the dependent variable (label) that we'll try to predict.

In [ ]:
df2['HIGH_EI'] = (df2['EnvironmentalIntensity(Sales)'].gt(df2['EnvironmentalIntensity(Sales)'].median())).astype(int)

In [ ]:
!pip install transformers
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 2.5MB 3.0MB/s 
     |████████████████████████████████| 3.3MB 16.7MB/s 
     |████████████████████████████████| 901kB 39.0MB/s 


Load a pre-trained BERT model.

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Tokenize the textual data for DistilBERT.

In [ ]:
tokenized = df2['description'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Pad all lists of tokenized values to the same size

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
np.array(padded).shape

(41, 463)

Create attention mask variable for BERT to ignore (mask) the padding when it's processing its input.

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(41, 463)

We run the pretrained DistilBERT model on the prepared predictor and keep the result in last_hidden_states variable.

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

Keep the first layer of the hidden states and assign the outcome variable to labels.

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
labels = df2['HIGH_EI']

Split the data in train and test subsets, train the Logistic Regression on train set and evaluate its accuracy on the test set.

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels,test_size=0.2,random_state=42)
lr_clf = LogisticRegression(max_iter=5000)
lr_clf.fit(train_features, train_labels)
print(lr_clf.score(test_features, test_labels))

0.6666666666666666


Summary: Our model can 67% accuratly capture whether the company is high or low environmental intensity.

In [ ]:
test_labels

24    0
13    0
8     0
25    1
4     1
40    0
19    1
39    0
29    1
Name: HIGH_EI, dtype: int64